In [1]:
from bs4 import BeautifulSoup  
import requests
import pandas as pd 

In [180]:
Genres=["Action","Action-Adventure","Adventure","Board+Game","Education","Fighting","Misc","MMO","Party","Music","Platform","Puzzle","Racing","Role-Playing","Sandbox","Shooter","Simulation","Sports","Strategy","Visual+Novel"]

def Get_Games_Part1(genres,sfilter):
    #list for every column
    Title=[] 
    Platforms=[] 
    Publisher=[] 
    Total_sales=[]
    USA_Sales=[] 
    EUR_Sales=[] 
    Japan_Sales=[]
    Release_date=[] 
    Genre_game=[] 
    
    for genre in genres:
        url1=f"https://www.vgchartz.com/games/games.php?page=1&results=200&genre={genre}&order=TotalSales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=0"
        html1=requests.get(url1)
        soup1=BeautifulSoup(html1.content,"html.parser")
        
        pages=int(soup1.findAll("span")[-3].find("a").attrs["href"].split("=")[1].split("&")[0]) #all pages

        
        games_tags=[] 
        for p in range(0, pages):
            url=f"https://www.vgchartz.com/games/games.php?page={p}&results=200&genre={genre}&order=TotalSales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=0"
            html=requests.get(url)
            soup=BeautifulSoup(html.content,"html.parser")
            games_tags.extend(soup.findAll("tr",sfilter))
        
        for tag in games_tags:
            Title.append(tag.findAll("td")[2].text.strip())
            Platforms.append(tag.findAll("td")[3].find("img").attrs["alt"])
            Publisher.append(tag.findAll("td")[4].text.strip())
            Total_sales.append(tag.findAll("td")[6].text )
            USA_Sales.append(tag.findAll("td")[7].text)
            EUR_Sales.append(tag.findAll("td")[8].text)
            Japan_Sales.append(tag.findAll("td")[9].text )
            Release_date.append(tag.findAll("td")[11].text.strip() )
            Genre_game.append(genre)
            
            #Dictionary containing all the games   
            dic_games={"Title":Title,
                       "Platforms":Platforms,
                       "Publisher":Publisher,
                       "Genre": Genre_game,
                       "Total_sales":Total_sales,
                       "USA_Sales":USA_Sales,
                       "EUR_Sales":EUR_Sales,
                       "Japan_Sales":Japan_Sales,
                       "Release_date":Release_date}
            
    return dic_games



In [177]:
def df_games(dic):
    df=pd.DataFrame(dic)
    #Deleting the m from the coloumns
    df["Total_sales"]=df["Total_sales"].apply(lambda x :x.split("m")[0]) 
    df["USA_Sales"]=df["USA_Sales"].apply(lambda x :x.split("m")[0]) 
    df["EUR_Sales"]=df["EUR_Sales"].apply(lambda x :x.split("m")[0]) 
    df["Japan_Sales"]=df["Japan_Sales"].apply(lambda x :x.split("m")[0])
    #df["Release_date"]=df["Release_date"].replace(to_replace='N/A',value="100")
    #df["Release_date"] = df[df['Release_date'].dropna()]
    
    #Formating the date
    
    df["Release_date"]=df["Release_date"].apply(lambda x:" ".join(x.split("th")))
    df["Release_date"]=df["Release_date"].apply(lambda x:" ".join(x.split("st"))) #Deleting the st abreviation of the Date
    df["Year"]=df["Release_date"].apply(lambda x: ("20"+(x.split(" ")[-1]) if int(x.split(" ")[-1])<50 else "19"+ (x.split(" ")[-1])) if(x!="N/A") else "N/A" )
    return df

In [181]:
sf1={"style":"background-image:url(../imgs/chartBar_large.gif); height:70px"}
sf2={"style":"background-image:url(../imgs/chartBar_alt_large.gif); height:70px"}
df_tr_1=df_games(Get_Games_Part1(Genres,sf1))
df_tr_2=df_games(Get_Games_Part1(Genres,sf2))
df_all_games=pd.concat([df_tr_1,df_tr_2])#concating two dataframes
df_all_games.to_csv("Games_Data.csv",index=False)
#Concatenating the Two Dataframe into one 



In [182]:
r=pd.read_csv("Games_Data.csv")
clean=r.dropna()
clean


,Title,Platforms,Publisher,Genre,Total_sales,USA_Sales,EUR_Sales,Japan_Sales,Release_date,Year
0,Grand Theft Auto: San Andreas,PS2,Rockstar Games,Action,20.81,9.43,0.40,0.41,26 Oct 04,2004.0
1,Grand Theft Auto V,PS4,Rockstar Games,Action,19.39,6.06,9.71,0.60,18 Nov 14,2014.0
2,Grand Theft Auto V,X360,Rockstar Games,Action,15.86,9.06,5.33,0.06,17 Sep 13,2013.0
3,Grand Theft Auto IV Read the review,X360,Rockstar Games,Action,11.09,6.80,3.11,0.14,29 Apr 08,2008.0
4,Uncharted 4: A Thief's End,PS4,Sony Interactive Entertainment,Action,10.33,4.49,3.93,0.21,10 May 16,2016.0
...,...,...,...,...,...,...,...,...,...,...
63843,Tokyo Twilight Ghost Hunters,PS3,Aksys Games,Visual+Novel,0.07,0.03,0.00,0.02,10 Mar 15,2015.0
63844,Steins;Gate,PS3,5pb,Visual+Novel,0.06,0.01,0.02,0.02,18 Sep 15,2015.0
63936,Tears to Tiara II: Heir of the Overlord,PS3,NIS America,Visual+Novel,0.15,0.07,0.01,0.06,14 Oct 14,2014.0
63943,Tokyo Twilight Ghost Hunters,PS3,Aksys Games,Visual+Novel,0.07,0.03,0.00,0.02,10 Mar 15,2015.0
